 ### Question 1 (TIK TAK TOE with Alpha Beta Purning)

In [1]:
import numpy as np
import time
import threading
import pygame
import sys


pygame.init()

WIDTH, HEIGHT = 600, 600
LINE_WIDTH = 10
BOARD_ROWS, BOARD_COLS = 4, 4
CELL_SIZE = WIDTH // BOARD_COLS
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
MAX_DEPTH = 4  

class TicTacToe:
    def __init__(self, ai_first=True, use_alpha_beta=True):
        self.board = np.full((4, 4), '-')
        self.ai_first = ai_first
        self.use_alpha_beta = use_alpha_beta
        self.current_player = 'X' if ai_first else 'O'
        self.window = pygame.display.set_mode((WIDTH, HEIGHT))
        pygame.display.set_caption(" Tic-Tac-Toe 4x4 Grid")
        self.window.fill(WHITE)
        self.draw_grid()
        self.game_over = False
        self.nodes_expanded = 0
        if ai_first:
            threading.Thread(target=self.ai_move, daemon=True).start()
    
    def draw_grid(self):
        for row in range(1, BOARD_ROWS):
            pygame.draw.line(self.window, BLACK, (0, row * CELL_SIZE), (WIDTH, row * CELL_SIZE), LINE_WIDTH)
        for col in range(1, BOARD_COLS):
            pygame.draw.line(self.window, BLACK, (col * CELL_SIZE, 0), (col * CELL_SIZE, HEIGHT), LINE_WIDTH)
        pygame.display.update()
    
    def draw_move(self, row, col):
        center_x = col * CELL_SIZE + CELL_SIZE // 2
        center_y = row * CELL_SIZE + CELL_SIZE // 2
        if self.board[row, col] == 'X':
            pygame.draw.line(self.window, RED, (center_x - 50, center_y - 50), (center_x + 50, center_y + 50), LINE_WIDTH)
            pygame.draw.line(self.window, RED, (center_x + 50, center_y - 50), (center_x - 50, center_y + 50), LINE_WIDTH)
        else:
            pygame.draw.circle(self.window, BLACK, (center_x, center_y), 50, LINE_WIDTH)
        pygame.display.update()
    
    def draw_text(self, text, color=BLACK):
        font = pygame.font.SysFont(None, 60)
        text_surface = font.render(text, True, color)
        text_rect = text_surface.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        self.window.blit(text_surface, text_rect)
        pygame.display.update()
    
    def make_move(self, row, col):
        if self.board[row, col] == '-' and not self.game_over:
            # Start timing for the move
            start_time = time.time()
            
            self.board[row, col] = self.current_player
            self.draw_move(row, col)
            
            # End timing and calculate duration
            end_time = time.time()
            time_taken = end_time - start_time
            
            # Print move details (nodes_expanded is 0 for human)
        #    print(f"Human move: ({row}, {col})")
        #    print(f"Nodes expanded: 0")  # No search algorithm for human
        #    print(f"Time taken: {time_taken:.4f} seconds")
            
            if self.is_winner(self.current_player):
                self.draw_text(f"{self.current_player} Wins!", RED if self.current_player == 'X' else BLACK)
                self.game_over = True
            elif self.is_board_full():
                self.draw_text("Draw!", BLACK)
                self.game_over = True
            else:
                self.current_player = 'O' if self.current_player == 'X' else 'X'
                if self.current_player == 'X' and not self.game_over:
                    threading.Thread(target=self.ai_move, daemon=True).start()
    
    def is_winner(self, player):
        for row in self.board:
            if all(cell == player for cell in row):
                return True
        for col in range(4):
            if all(self.board[row][col] == player for row in range(4)):
                return True
        if all(self.board[i][i] == player for i in range(4)) or all(self.board[i][3 - i] == player for i in range(4)):
            return True
        return False
    
    def is_board_full(self):
        return '-' not in self.board.flatten()
    
    def minimax(self, depth, is_maximizing, alpha=-np.inf, beta=np.inf):
        self.nodes_expanded += 1
        if self.is_winner('X'):
            return 1
        if self.is_winner('O'):
            return -1
        if self.is_board_full() or depth >= MAX_DEPTH:
            return 0

        if is_maximizing:
            best_score = -np.inf
            for row in range(4):
                for col in range(4):
                    if self.board[row, col] == '-':
                        self.board[row, col] = 'X'
                        score = self.minimax(depth + 1, False, alpha, beta)
                        self.board[row, col] = '-'
                        best_score = max(score, best_score)
                        if self.use_alpha_beta:
                            alpha = max(alpha, best_score)
                            if beta <= alpha:
                                break
            return best_score
        else:
            best_score = np.inf
            for row in range(4):
                for col in range(4):
                    if self.board[row, col] == '-':
                        self.board[row, col] = 'O'
                        score = self.minimax(depth + 1, True, alpha, beta)
                        self.board[row, col] = '-'
                        best_score = min(score, best_score)
                        if self.use_alpha_beta:
                            beta = min(beta, best_score)
                            if beta <= alpha:
                                break
            return best_score
    
    def ai_move(self):
        start_time = time.time()
        self.nodes_expanded = 0
        best_score = -np.inf
        best_move = None
        for row in range(4):
            for col in range(4):
                if self.board[row, col] == '-':
                    self.board[row, col] = 'X'
                    score = self.minimax(0, False)
                    self.board[row, col] = '-'
                    if score > best_score:
                        best_score = score
                        best_move = (row, col)
        
        if best_move and not self.game_over:
            end_time = time.time()
            self.make_move(best_move[0], best_move[1])
            print(f"AI move: {best_move}")
            print(f"Nodes expanded: {self.nodes_expanded}")
            print(f"Time taken: {end_time - start_time:.4f} seconds")

if __name__ == "__main__":
    ai_first = input("Should AI play first ?  (y/n): ").strip().lower() == 'y'
    use_alpha_beta = input("Press Y for Alpha-Beta Pruning and N for MinMax  (y/n): ").strip().lower() == 'y'
    
    game = TicTacToe(ai_first, use_alpha_beta)
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.MOUSEBUTTONDOWN and game.current_player == 'O' and not game.game_over:
                x, y = event.pos
                row = y // CELL_SIZE
                col = x // CELL_SIZE
                game.make_move(row, col)
        pygame.display.update()
    pygame.quit()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
AI move: (0, 0)
Nodes expanded: 36324
Time taken: 1.1348 seconds
AI move: (0, 2)
Nodes expanded: 24680
Time taken: 0.7668 seconds


### Question 3 (Hill climb )

In [ ]:
import pygame
import numpy as np
import random
import time

pygame.init()

WIDTH, HEIGHT = 600, 600
BOARD_SIZE = 8
CELL_SIZE = WIDTH // BOARD_SIZE
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# initlizzae the queen img
QUEEN_IMG = pygame.image.load("queen.png")
QUEEN_IMG = pygame.transform.scale(QUEEN_IMG, (CELL_SIZE, CELL_SIZE))
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("   8-Queens problem  - ( Hill Climbing ) ")

def count_conflicts(board):
    conflicts = 0
    for i in range(BOARD_SIZE):
        for j in range(i+1, BOARD_SIZE):
            if board[i] == board[j] or abs(i-j) == abs(board[i]-board[j]):
                conflicts += 1
    return conflicts

def generate_initial_board():
    return np.random.randint(0, BOARD_SIZE, BOARD_SIZE)

def get_best_neighbor(board):
    best_board = None
    # tell conflict
    min_conflicts = count_conflicts(board)
    # 
    for row in range(BOARD_SIZE):
        original_col = board[row]
        for new_col in range(BOARD_SIZE):
            if new_col != original_col:
                neighbor = np.copy(board)
                neighbor[row] = new_col
                current_conflicts = count_conflicts(neighbor)
                if current_conflicts < min_conflicts:
                    best_board = np.copy(neighbor)
                    min_conflicts = current_conflicts
                elif current_conflicts == min_conflicts and random.random() < 0.5:
                    best_board = np.copy(neighbor)
    
    return best_board, min_conflicts

def draw_board(board):
    screen.fill(WHITE)
    for row in range(BOARD_SIZE):
        for col in range(BOARD_SIZE):
            if (row + col) % 2 == 0:
                pygame.draw.rect(screen, BLACK, (col*CELL_SIZE, row*CELL_SIZE, CELL_SIZE, CELL_SIZE))
            if board[row] == col:
                screen.blit(QUEEN_IMG, (col*CELL_SIZE, row*CELL_SIZE))
    pygame.display.update()
    time.sleep(0.5)

def hill_climbing():
    board = generate_initial_board()
    current_conflicts = count_conflicts(board)
    restarts = 0

    while current_conflicts > 0:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return

        best_neighbor, new_conflicts = get_best_neighbor(board)
        
        if best_neighbor is None or new_conflicts >= current_conflicts:
            board = generate_initial_board()
            current_conflicts = count_conflicts(board)
            restarts += 1
            print(f"Restart #{restarts}")
        else:
            board = best_neighbor
            current_conflicts = new_conflicts
        
        draw_board(board)
    
    print(" hurrrrrrrr.  the Solution founded ")
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return

hill_climbing()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
Solution found!


### Question 3 (Local Beam Search  )

In [5]:
import pygame
import numpy as np
import random
import time

pygame.init()

WIDTH, HEIGHT = 600, 600
BOARD_SIZE = 8
CELL_SIZE = WIDTH // BOARD_SIZE
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BEAM_WIDTH = 8
QUEEN_IMG = pygame.image.load("queen.png")
QUEEN_IMG = pygame.transform.scale(QUEEN_IMG, (CELL_SIZE, CELL_SIZE))
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("8-Queens problem - ( Local Beam Search ) ")

def count_conflicts(board):
    conflicts = 0
    for i in range(BOARD_SIZE):
        for j in range(i+1, BOARD_SIZE):
            if board[i] == board[j] or abs(i-j) == abs(board[i]-board[j]):
                conflicts += 1
    return conflicts

def generate_initial_boards(k):
    return [np.random.randint(0, BOARD_SIZE, BOARD_SIZE) for _ in range(k)]

def get_all_neighbors(boards):
    neighbors = []
    for board in boards:
        for row in range(BOARD_SIZE):
            original_col = board[row]
            for new_col in range(BOARD_SIZE):
                if new_col != original_col:
                    new_board = np.copy(board)
                    new_board[row] = new_col
                    neighbors.append(new_board)
    return neighbors
# function to visulated the board
def draw_board(board):
    screen.fill(WHITE)
    for row in range(BOARD_SIZE):
        for col in range(BOARD_SIZE):
            if (row + col) % 2 == 0:
                pygame.draw.rect(screen, BLACK, (col*CELL_SIZE, row*CELL_SIZE, CELL_SIZE, CELL_SIZE))
            if board[row] == col:
                screen.blit(QUEEN_IMG, (col*CELL_SIZE, row*CELL_SIZE))
    pygame.display.update()
    time.sleep(0.2)
# main algorithm
def local_beam(beam_width=BEAM_WIDTH):
    current_boards = generate_initial_boards(beam_width)
    bst_board = None
    best_conflicts = float('inf')

    while best_conflicts > 0:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return

        all_neighbors = get_all_neighbors(current_boards)
        if not all_neighbors:
            current_boards = generate_initial_boards(beam_width)
            all_neighbors = get_all_neighbors(current_boards)
        
        
        scored_neighbors = [(count_conflicts(board), board) for board in all_neighbors]
        scored_neighbors.sort(key=lambda x: x[0])
        
        current_boards = [board for (score, board) in scored_neighbors[:beam_width]]
        
        #updating the  best solution
        current_best = min(current_boards, key=lambda x: count_conflicts(x))
        current_best_conflicts = count_conflicts(current_best)
        
        if current_best_conflicts < best_conflicts:
            best_conflicts = current_best_conflicts
            bst_board = current_best
        
        draw_board(bst_board)
    
    print("hurrrrr , the  Solution founded ")
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return

local_beam()

hurrrrr , the  Solution founded 


### Question 3 (Stimulated Annealing )

In [9]:
import pygame
import numpy as np
import random
import math
import time
import sys

pygame.init()

WIDTH, HEIGHT = 600, 600
BOARD_SIZE = 8
CELL_SIZE = WIDTH // BOARD_SIZE
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
QUEEN_IMG = pygame.image.load("queen.png")
QUEEN_IMG = pygame.transform.scale(QUEEN_IMG, (CELL_SIZE, CELL_SIZE))
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("8-Queens problem - ( Local Search Algorithm )")

def count_conflicts(board):
    conflicts = 0
    for i in range(BOARD_SIZE):
        for j in range(i + 1, BOARD_SIZE):
            if board[i] == board[j] or abs(i - j) == abs(board[i] - board[j]):
                conflicts += 1
    return conflicts

def generate_initial_board():
    return np.random.randint(0, BOARD_SIZE, BOARD_SIZE)

def get_best_neighbor(board):
    current_conflicts = count_conflicts(board)
    best_conflicts = current_conflicts
    best_neighbors = []

    for row in range(BOARD_SIZE):
        original_col = board[row]
        for new_col in range(BOARD_SIZE):
            if new_col == original_col:
                continue
            neighbor = np.copy(board)
            neighbor[row] = new_col
            conflicts = count_conflicts(neighbor)

            if conflicts < best_conflicts:
                best_conflicts = conflicts
                best_neighbors = [np.copy(neighbor)]
            elif conflicts == best_conflicts:
                best_neighbors.append(np.copy(neighbor))

    if not best_neighbors:
        return None, current_conflicts

    return random.choice(best_neighbors), best_conflicts

def draw_board(board):
    screen.fill(WHITE)
    for row in range(BOARD_SIZE):
        for col in range(BOARD_SIZE):
            if (row + col) % 2 == 0:
                pygame.draw.rect(screen, BLACK, (col * CELL_SIZE, row * CELL_SIZE, CELL_SIZE, CELL_SIZE))
            if board[row] == col:
                screen.blit(QUEEN_IMG, (col * CELL_SIZE, row * CELL_SIZE))
    pygame.display.update()
    time.sleep(0.5)



def simuulated_anneling():
    board = generate_initial_board()
    crt_conflict = count_conflicts(board)
    temp = 1000.0
    cooling_rate = 0.99
    steps = 0

    running = True
    while running and temp > 0.1 and crt_conflict > 0:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                pygame.quit()
                sys.exit()

        row = random.randint(0, BOARD_SIZE - 1)
        new_col = random.randint(0, BOARD_SIZE - 1)
        while new_col == board[row]:
            new_col = random.randint(0, BOARD_SIZE - 1)
        neighbor = np.copy(board)
        neighbor[row] = new_col
        new_confloct = count_conflicts(neighbor)
        delta = new_confloct - crt_conflict

        if delta < 0 or random.random() < math.exp(-delta / temp):
            board = neighbor
            crt_conflict = new_confloct

        temp *= cooling_rate
        steps += 1
        draw_board(board)
        time.sleep(0.1)

    if crt_conflict == 0:
        print(" hurrrrr. Solution founded")
    else:
        print("Stopped without finding a solution.")
    draw_board(board)
    time.sleep(2)
    pygame.quit()
    sys.exit()

if __name__ == "__main__":

    print("Running Simulated Annealing...")
    simuulated_anneling()
    pygame.quit()
    sys.exit()

Running Simulated Annealing...


SystemExit: 

c:\semester_4\Data_Science\venv\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Question 2 (Checker With alpha beta and minMax )

In [5]:
import numpy as np
import pygame
import sys
import copy
import time  

# Setting the grid system
WIDTH, HEIGHT = 600, 600
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
CELL_SIZE = WIDTH // 8
KING_RED = (200, 0, 0)
KING_BLUE = (0, 0, 200)

class Checkers:
    def __init__(self, ai_depth=3, use_alpha_beta=False, ai_player='B'):  
        self.board = self.create_board()
        self.window = pygame.display.set_mode((WIDTH, HEIGHT))
        pygame.display.set_caption("Checkers")
        self.selected_piece = None
        self.current_player = 'R'
        self.ai_depth = ai_depth
        self.use_alpha_beta = use_alpha_beta
        self.ai_player = ai_player
        self.nodes_expanded = 0  # Track nodes expanded
        self.draw_board()
    
    def create_board(self):
        board = np.full((8, 8), '-', dtype='<U2')
        for row in range(3):
            for col in range(8):
                if (row + col) % 2 == 1:
                    board[row][col] = 'B'
        for row in range(5, 8):
            for col in range(8):
                if (row + col) % 2 == 1:
                    board[row][col] = 'R'
        return board
    
    def get_directions(self, piece):
        if piece == 'R':
            return [(-1, -1), (-1, 1)]
        elif piece == 'B':
            return [(1, -1), (1, 1)]
        elif piece in ['RK', 'BK']:
            return [(-1, -1), (-1, 1), (1, -1), (1, 1)]
        return []
    
    def is_enemy(self, piece, target):
        if piece in ['R', 'RK']:
            return target in ['B', 'BK']
        elif piece in ['B', 'BK']:
            return target in ['R', 'RK']
        return False
    
    def valid_moves(self, row, col):
        captures = []
        moves = []
        piece = self.board[row][col]
        if piece not in ['R', 'B', 'RK', 'BK']:
            return []
        directions = self.get_directions(piece)
        
        for drow, dcol in directions:
            adj_row, adj_col = row + drow, col + dcol
            if 0 <= adj_row < 8 and 0 <= adj_col < 8:
                target = self.board[adj_row][adj_col]
                if self.is_enemy(piece, target):
                    jump_row, jump_col = adj_row + drow, adj_col + dcol
                    if 0 <= jump_row < 8 and 0 <= jump_col < 8 and self.board[jump_row][jump_col] == '-':
                        captures.append((jump_row, jump_col))
        
        if captures:
            return captures
        
        for drow, dcol in directions:
            new_row, new_col = row + drow, col + dcol
            if 0 <= new_row < 8 and 0 <= new_col < 8 and self.board[new_row][new_col] == '-':
                moves.append((new_row, new_col))
        return moves
    
    def draw_board(self):
        self.window.fill(WHITE)
        for row in range(8):
            for col in range(8):
                color = BLACK if (row + col) % 2 == 0 else WHITE
                pygame.draw.rect(self.window, color, (col * CELL_SIZE, row * CELL_SIZE, CELL_SIZE, CELL_SIZE))
                piece = self.board[row][col]
                if piece in ['R', 'B']:
                    pygame.draw.circle(self.window, RED if piece == 'R' else BLUE,
                                     (col * CELL_SIZE + CELL_SIZE//2, row * CELL_SIZE + CELL_SIZE//2), CELL_SIZE//2-5)
                elif piece in ['RK', 'BK']:
                    pygame.draw.circle(self.window, KING_RED if piece == 'RK' else KING_BLUE,
                                     (col * CELL_SIZE + CELL_SIZE//2, row * CELL_SIZE + CELL_SIZE//2), CELL_SIZE//2-5)
        pygame.display.update()
    
    def move_piece(self, from_row, from_col, to_row, to_col):
        valid_moves = self.valid_moves(from_row, from_col)
        if (to_row, to_col) not in valid_moves:
            return False
        
        original_player = self.current_player
        
        piece = self.board[from_row][from_col]
        self.board[to_row][to_col] = piece
        self.board[from_row][from_col] = '-'

        if abs(to_row - from_row) == 2:
            mid_row = (from_row + to_row) // 2
            mid_col = (from_col + to_col) // 2
            self.board[mid_row][mid_col] = '-'

        if (to_row == 0 and piece == 'R') or (to_row == 7 and piece == 'B'):
            self.board[to_row][to_col] = piece + 'K'

        self.current_player = 'B' if self.current_player == 'R' else 'R'

        if not any('B' in row for row in self.board):
            print("You (Red) wins!")
            pygame.quit()
            sys.exit()
        if not any('R' in row for row in self.board):
            print("AI (Blue) wins!")
            pygame.quit()
            sys.exit()
            
        if not self.get_all_moves(self.board, self.current_player):
            winner = 'Red' if original_player == 'R' else 'Blue'
            print(f"{winner} wins by immobilization!")
            pygame.quit()
            sys.exit()

        return True
    
    def get_all_moves(self, board, player):
        moves = []
        for row in range(8):
            for col in range(8):
                piece = board[row][col]
                if piece in [player, player + 'K']:
                    piece_moves = self.valid_moves(row, col)
                    for move in piece_moves:
                        moves.append(((row, col), move))
        return moves
    
    def simulate_move(self, board, move):
        new_board = copy.deepcopy(board)
        (from_row, from_col), (to_row, to_col) = move
        piece = new_board[from_row][from_col]
        new_board[to_row][to_col] = piece
        new_board[from_row][from_col] = '-'

        if abs(to_row - from_row) == 2:
            mid_row = (from_row + to_row) // 2
            mid_col = (from_col + to_col) // 2
            new_board[mid_row][mid_col] = '-'

        if (to_row == 0 and piece == 'R') or (to_row == 7 and piece == 'B'):
            new_board[to_row][to_col] = piece + 'K'

        return new_board
    
    def evaluate_board(self, board):
        score = 0
        for row in board:
            for cell in row:
                if cell == 'B':
                    score += 1
                elif cell == 'BK':
                    score += 3
                elif cell == 'R':
                    score -= 1
                elif cell == 'RK':
                    score -= 3
        return score
    
    def minimax_dls(self, board, depth, maximizing_player):
        self.nodes_expanded += 1  # Track nodes expanded
        if depth == 0 or self.is_terminal(board):
            return self.evaluate_board(board)
        
        if maximizing_player:
            max_eval = -float('inf')
            for move in self.get_all_moves(board, self.ai_player):
                new_board = self.simulate_move(board, move)
                eval = self.minimax_dls(new_board, depth-1, False)
                max_eval = max(max_eval, eval)
            return max_eval
        else:
            min_eval = float('inf')
            opponent = 'R' if self.ai_player == 'B' else 'B'
            for move in self.get_all_moves(board, opponent):
                new_board = self.simulate_move(board, move)
                eval = self.minimax_dls(new_board, depth-1, True)
                min_eval = min(min_eval, eval)
            return min_eval

    def alpha_beta(self, board, depth, alpha, beta, maximizing_player):
        self.nodes_expanded += 1  # Track nodes expanded
        if depth == 0 or self.is_terminal(board):
            return self.evaluate_board(board)
        
        if maximizing_player:
            max_eval = -float('inf')
            for move in self.get_all_moves(board, self.ai_player):
                new_board = self.simulate_move(board, move)
                eval = self.alpha_beta(new_board, depth-1, alpha, beta, False)
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return max_eval
        else:
            min_eval = float('inf')
            opponent = 'R' if self.ai_player == 'B' else 'B'
            for move in self.get_all_moves(board, opponent):
                new_board = self.simulate_move(board, move)
                eval = self.alpha_beta(new_board, depth-1, alpha, beta, True)
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return min_eval
    
    def is_terminal(self, board):
        return not any('B' in row for row in board) or not any('R' in row for row in board)
    
    def ai_move(self):
        start_time = time.time()  # Start timing
        self.nodes_expanded = 0  # Reset node counter
        best_move = None
        best_value = -float('inf')
        all_moves = self.get_all_moves(self.board, self.ai_player)
        
        for move in all_moves:
            new_board = self.simulate_move(self.board, move)
            if self.use_alpha_beta:
                value = self.alpha_beta(new_board, self.ai_depth-1, -float('inf'), float('inf'), False)
            else:
                value = self.minimax_dls(new_board, self.ai_depth-1, False)
            
            if value > best_value:
                best_value = value
                best_move = move
        
        if best_move:
            self.move_piece(*best_move[0], *best_move[1])
            self.draw_board()
            end_time = time.time()  # End timing
            print(f"AI Move: {best_move}")
            print(f"Nodes Expanded: {self.nodes_expanded}")
            print(f"Execution Time: {end_time - start_time:.4f} seconds")
            time.sleep(0.5)  # Add delay to see AI moves
        else:
            print("AI has no valid moves! You win!")
            pygame.quit()
            sys.exit()

if __name__ == "__main__":
    print("In Checkers AI Settings:")
    ai_depth = int(input("Please enter AI depth limit (1-5 recommended): "))
    use_ab = input("Press y for Alpha-Beta Pruning and n for MinMax (y/n)? ").lower() == 'y'
    
    pygame.init()
    game = Checkers(ai_depth=ai_depth, use_alpha_beta=use_ab, ai_player='B')
    running = True
    
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.MOUSEBUTTONDOWN and game.current_player != game.ai_player:
                x, y = event.pos
                col = x // CELL_SIZE
                row = y // CELL_SIZE
                if game.selected_piece is None:
                    if game.board[row][col] in [game.current_player, game.current_player + 'K']:
                        game.selected_piece = (row, col)
                else:
                    from_row, from_col = game.selected_piece
                    if game.move_piece(from_row, from_col, row, col):
                        game.draw_board()
                    game.selected_piece = None
        
        if game.current_player == game.ai_player and running:
            game.ai_move()
        
        pygame.display.update()
        time.sleep(0.1)  
    pygame.quit()

In Checkers AI Settings:
AI Move: ((2, 1), (3, 0))
Nodes Expanded: 63
Execution Time: 0.0113 seconds
AI Move: ((1, 0), (2, 1))
Nodes Expanded: 71
Execution Time: 0.0115 seconds
AI Move: ((2, 1), (4, 3))
Nodes Expanded: 68
Execution Time: 0.0040 seconds
AI Move: ((0, 1), (1, 0))
Nodes Expanded: 64
Execution Time: 0.0030 seconds
AI Move: ((1, 0), (2, 1))
Nodes Expanded: 72
Execution Time: 0.0075 seconds
AI Move: ((2, 1), (3, 2))
Nodes Expanded: 54
Execution Time: 0.0040 seconds
AI Move: ((1, 2), (2, 1))
Nodes Expanded: 48
Execution Time: 0.0056 seconds
AI Move: ((3, 0), (5, 2))
Nodes Expanded: 54
Execution Time: 0.0081 seconds
AI Move: ((2, 1), (3, 0))
Nodes Expanded: 49
Execution Time: 0.0095 seconds
AI Move: ((2, 5), (3, 6))
Nodes Expanded: 63
Execution Time: 0.0065 seconds
AI Move: ((1, 4), (3, 6))
Nodes Expanded: 56
Execution Time: 0.0057 seconds
AI Move: ((0, 3), (2, 1))
Nodes Expanded: 62
Execution Time: 0.0141 seconds
AI Move: ((0, 5), (1, 4))
Nodes Expanded: 64
Execution Time: 0.

SystemExit: 

c:\semester_4\Data_Science\venv\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
